In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName('OlistData') \
.getOrCreate()

25/09/20 14:15:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
hdfs_path = '/data/olist/'

In [3]:
spark

In [4]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True,inferSchema=True)

In [5]:
# Cache Frequently used Data for Better Performance

orders_df.cache()
customers_df.cache()
order_item_df.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [6]:
orders_items_joined_df = orders_df.join(order_item_df,'order_id','inner')

In [7]:
orders_items_products_df = orders_items_joined_df.join(products_df,'product_id','inner')

In [8]:
orders_items_products_sellers_df = orders_items_products_df.join(sellers_df,'seller_id','inner')

In [97]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

In [98]:
# GEolocation Data

full_orders_df = full_orders_df.join(geolocation_df,full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [99]:
full_orders_df = full_orders_df.join(reviews_df,'order_id','left')

In [100]:
full_orders_df = full_orders_df.join(payments_df,'order_id','left')

In [101]:
full_orders_df.cache()

25/09/20 14:45:16 WARN CacheManager: Asked to cache already cached data.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [14]:
from pyspark.sql.functions import *

In [102]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

# Total Revenues Per Seller

In [103]:
# Total Revenues Per Seller


seller_revenue_df = full_orders_df.groupBy('seller_id').agg(sum('price'))

In [104]:
seller_revenue_df.show(5)

+--------------------+------------------+
|           seller_id|        sum(price)|
+--------------------+------------------+
|3d5d0dc7073a299e3...|          170639.6|
|2138ccb85b11a4ec1...|1943866.0699999987|
|d650b663c3b5f6fb3...|         2253869.1|
|cd06602b43d8800bd...|353150.98000000045|
|3c487ae8f8d7542be...|1618845.7000000053|
+--------------------+------------------+
only showing top 5 rows



In [105]:
# total orders per unique customer

orders_by_customer = (
    full_orders_df.groupBy("customer_unique_id")
      .agg(countDistinct("order_id").alias("total_orders"))
      .orderBy(desc("total_orders"))
)

orders_by_customer.show(4)


+--------------------+------------+
|  customer_unique_id|total_orders|
+--------------------+------------+
|8d50f5eadf50201cc...|          16|
|3e43e6105506432c9...|           9|
|6469f99c1f9dfae77...|           7|
|ca77025e7201e3b30...|           7|
+--------------------+------------+
only showing top 4 rows



In [106]:
# Average Review Score Per Seller
avg_review_per_seller = (
    full_orders_df.groupBy("seller_id")
      .agg(avg("review_score").alias("avg_review_score"))
      .orderBy(desc("avg_review_score"))
)
# Most Sold Products ( Top 10 )
# Top Custoemrs By Spending

In [20]:
avg_review_per_seller.show()

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|51e0557bc7b86de13...|             5.0|
|94ca168e8bcb407ab...|             5.0|
|7cb946b1ad19faebe...|             5.0|
|78813699ffac347fe...|             5.0|
|9c1c0c36cd23c2089...|             5.0|
|b76f4d90e85657a24...|             5.0|
|762fafdaa57a532ee...|             5.0|
|13e85aac53340018b...|             5.0|
|f5b84683a9bf9e1df...|             5.0|
|10264f60a8f0a4d2f...|             5.0|
|d263fa444c1504a75...|             5.0|
|dd88d13bea9e4c2bf...|             5.0|
|fc4751cfe3b279c41...|             5.0|
|3bfba5a77604f6dfa...|             5.0|
|6548c2550b6e2d8f1...|             5.0|
|f90f77ef2799a27f8...|             5.0|
|a213f1bd6458f4a15...|             5.0|
|d8b8f2cf9ff6ba038...|             5.0|
|9d213f303afae4983...|             5.0|
|a20d8058c866dbaec...|             5.0|
+--------------------+----------------+
only showing top 20 rows



# Optimized Joins For Data integration

In [108]:
from pyspark.sql.functions import *

In [109]:
orders_items_joined_df = orders_df.join(order_item_df,'order_id','inner')

In [110]:
orders_items_products_df = orders_items_joined_df.join(products_df,'product_id','inner')

In [111]:
orders_items_products_sellers_df = orders_items_products_df.join(broadcast(sellers_df),'seller_id','inner')

In [112]:
full_orders_df = orders_items_products_sellers_df.join(customers_df,'customer_id','inner')

In [113]:
# GEolocation Data

full_orders_df = full_orders_df.join(broadcast(geolocation_df),full_orders_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [114]:
full_orders_df = full_orders_df.join(broadcast(reviews_df),'order_id','left')

In [115]:
full_orders_df = full_orders_df.join(payments_df,'order_id','left')

In [116]:
full_orders_df.cache()

25/09/20 14:45:54 WARN CacheManager: Asked to cache already cached data.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

# Aggregation

In [117]:
# Total Orders Per Customer

customer_order_count_df = full_orders_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_orders'))\
.orderBy(desc('total_orders'))
         
customer_order_count_df.show(5)

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
+--------------------+------------+
only showing top 5 rows



In [118]:
#Average Review Score Per Seller

seller_review_df = full_orders_df.groupBy('seller_id')\
.agg(avg('review_score').alias('avg_review_score'))\
.orderBy(desc('avg_review_score'))

seller_review_df.show()

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|26b482dccfa29bd2e...|             5.0|
|59de1d4c8c057c2a5...|             5.0|
|661ad5fcc28cdfb8b...|             5.0|
|b9c0ffa92f3ff9a5a...|             5.0|
|1927cdb9fa74f5270...|             5.0|
|93bef6edaa84956e8...|             5.0|
|7b3447b497e55af8c...|             5.0|
|297d5eccd19fa9a83...|             5.0|
|75fdd0a732287f949...|             5.0|
|0ad80de75c8113263...|             5.0|
|16bdc8cefd0e32a6f...|             5.0|
|5f5a58930c3c35f3b...|             5.0|
|9e94fa26a70ede4d2...|             5.0|
|28a26ebce31d5918b...|             5.0|
|c8665a4dd081a2c43...|             5.0|
|4450bd8327d471c4a...|             5.0|
|4049512728d969bec...|             5.0|
|ab27bbbad5239bc31...|             5.0|
|c8c1bea22194a4eef...|             5.0|
|9b76ad3d04236f76b...|             5.0|
+--------------------+----------------+
only showing top 20 rows



In [119]:
# Top 10 Most Sold Products

top_products_df = full_orders_df.groupBy('product_id')\
.agg(count('order_id').alias('total_sold'))\
.orderBy(desc('total_sold'))\
.limit(10)

top_products_df.show()

+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|     86740|
|422879e10f4668299...|     81110|
|99a4788cb24856965...|     78775|
|389d119b48cf3043d...|     60248|
|d1c427060a0f73f6b...|     59274|
|368c6c730842d7801...|     58358|
|53759a2ecddad2bb8...|     52654|
|53b36df67ebb7c415...|     52105|
|154e7e31ebfa09220...|     42700|
|3dd2a17168ec895c7...|     40787|
+--------------------+----------+



In [121]:
# top 10 customer by spending 
from pyspark.sql.functions import sum

total_customer_spending = full_orders_df.groupby('customer_id').agg(sum('payment_value').alias('total_spending'))

In [122]:
total_customer_spending.orderBy(desc('total_spending')).show()

+--------------------+--------------------+
|         customer_id|      total_spending|
+--------------------+--------------------+
|1ff773612ab8934db...| 1.756825199999893E7|
|05455dfa7cd02f13d...|1.3282083359999327E7|
|ec5b2ba62e5743423...|1.0388528640000112E7|
|0c792d32a3251b4f6...|   8254681.600000529|
|78fc46047c4a639e8...|   7488519.999999339|
|1617b1357756262bf...|   7433259.520000033|
|1dbc055ccab23ed89...|   7216273.400000708|
|d5f2b3f597c7ccafb...|   6800018.119998923|
|dd3f1762eb601f41c...|  6746388.4800006235|
|10de381f8a8d23fff...|   5184499.500000076|
|30bb84b541c96af98...|   4740404.549999884|
|d72181923840c8895...|   4513322.700000229|
|e7d6802668de6e74d...|  4000041.4000000926|
|cb87122c4871e2027...|  3957404.0000001485|
|a972623b3481cbfd9...|  3716577.3600002443|
|df55c14d1476a9a34...|   3678102.619999968|
|f7622098214b4634b...|   3657923.519999952|
|cc803a2c412833101...|           3387471.0|
|f4d64a735d1f90f13...|   3224595.419999714|
|6361b9f3b85d41860...|   3207389

# Window Function and Ranking

In [123]:
from pyspark.sql.window import Window

In [124]:

# Dense Rank for Sellers Based on Revenue

window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

In [125]:
# Rank Top Selling Products Per seller

top_seller_products_df = full_orders_df.withColumn('rank',rank().over(window_spec)).filter(col('rank')<=5)

top_seller_products_df.select('seller_id','price','rank').show(5)

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
+--------------------+-----+----+
only showing top 5 rows



In [38]:

# Dense Rank for Sellers Based on Revenue


In [126]:
top_dense_rank_seller = full_orders_df.withColumn('dense_rank',dense_rank().over(window_spec))

In [127]:
top_dense_rank_seller.select('seller_id','price','dense_rank').show()

+--------------------+-----+----------+
|           seller_id|price|dense_rank|
+--------------------+-----+----------+
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
|0015a82c2db000af6...|895.0|         1|
+--------------------+-----+----------+
only showing top 20 rows



# Advance Aggregation and Enrichment

In [128]:
full_orders_df

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [129]:
# Total Revenue & Average Order Value (AOV) per Customer

customer_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_spent'),
    round(avg('price'),2).alias('AOV')
)\
.orderBy(desc('total_spent'))


customer_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spent|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [43]:
6662844/6876

969.0

In [130]:
# Seller Performance Metrics ( Revenue, Average Review, Order Count)

seller_performance_df = full_orders_df.groupBy('seller_id') \
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('review_score'),2).alias('avg_review_score'),
    round(stddev('price'),2).alias('price_variability')
)\
.orderBy(desc('total_revenue'))

In [45]:
seller_performance_df.show()

+--------------------+------------+--------------------+----------------+-----------------+
|           seller_id|total_orders|       total_revenue|avg_review_score|price_variability|
+--------------------+------------+--------------------+----------------+-----------------+
|4869f7a5dfa277a7d...|      184587|3.6138717319999985E7|            4.09|           111.65|
|53243585a1d6dc264...|       54514| 3.429159294999998E7|            4.12|           499.65|
|4a3ca9315b744ce9f...|      330661| 3.375957084000011E7|            3.77|            59.37|
|7c67e1448b00f6e96...|      233306|3.2282321789999757E7|            3.42|            50.39|
|fa1c13f2614d7b5c4...|       87686|3.0139386309999965E7|            4.38|            307.7|
|da8622b14eb17ae28...|      264433|2.9857669730000056E7|            3.98|            72.92|
|7e93a43ef30c4f03f...|       50226|2.6315706299999937E7|            4.15|           377.24|
|1025f0e2d44d7041d...|      229587| 2.293751851999998E7|            3.89|       

In [131]:
# Product Popularity Metrics

product_metrics_df = full_orders_df.groupBy('product_id')\
.agg(
    count('order_id').alias('total_sales'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_price'),
    round(stddev('price'),2).alias('price_volatility'),\
    collect_set('seller_id').alias('unique_sellers')    
)\
.orderBy(desc('total_sales'))

In [132]:
product_metrics_df.show()

+--------------------+-----------+------------------+---------+----------------+--------------------+
|          product_id|total_sales|     total_revenue|avg_price|price_volatility|      unique_sellers|
+--------------------+-----------+------------------+---------+----------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740| 6164630.300000017|    71.07|            3.17|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4442791.510000012|    54.77|            4.46|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775|  6921762.71000002|    87.87|            4.08|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248|3280533.1300000106|    54.45|            4.37|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274| 8220103.329999992|   138.68|           16.58|[a1043bafd471dff5...|
|368c6c730842d7801...|      58358|3181698.8999999925|    54.52|            4.59|[1f50f920176fa81d...|
|53759a2ecddad2bb8...|      52654|2893017.4999999977|    54.94|            4.52|[1

In [139]:
# # Monthly Revenue and Order Count Trend ----> HW

# order_purchase_timestamp ---> month

# monthly_revenue_df = full_orders_df.groupBy('total_orders')

# total_orders 
# total_revenue
# avg_order_value
# min_order_value
# max_orderValues

In [133]:
# Customer Retention Analysis ( First & Last Order )

customer_retention_df = full_orders_df.groupBy('customer_id')\
.agg(
    first('order_purchase_timestamp').alias('first_order_date'),
    last('order_purchase_timestamp').alias('last_order_date'),
    count('order_id').alias('total_orders'),
    round(avg('price'),2).alias('aov')
)\
.orderBy(desc('total_orders'))

In [134]:
customer_retention_df.show()

+--------------------+-------------------+-------------------+------------+------+
|         customer_id|   first_order_date|    last_order_date|total_orders|   aov|
+--------------------+-------------------+-------------------+------------+------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427| 85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752| 43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|  26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001| 36.59|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876| 12.49|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876| 969.0|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|  59.0|
|c2f18647725395af4...|2018-03-06 19:21:47|2018-03-06 19:21:47|        6612|  34.9|
|24e7dc2ff8c071263...|2017-11-24 16:16:45|2017-11-24 16:16:45|        6597|  59.2|
|7bb

# Extended Enrichment 

In [135]:
# Order Status Flags

full_orders_df = full_orders_df.withColumn('is_delivered', when(col('order_status')== 'delivered',lit(1)).otherwise(lit(0)))\
.withColumn('is_canceled', when(col('order_status')== 'canceled',lit(1)).otherwise(lit(0)))

In [52]:
full_orders_df.where(full_orders_df['order_status']=='canceled').select('order_status','is_delivered','is_canceled').show(100)

+------------+------------+-----------+
|order_status|is_delivered|is_canceled|
+------------+------------+-----------+
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|
|    canceled|           0|          1|


In [136]:
# Order Revenue Calcualtion

full_orders_df = full_orders_df.withColumn('order_revenue',col('price')+col('freight_value'))

In [137]:
full_orders_df.select('price','freight_value','order_revenue').show()

+-----+-------------+------------------+
|price|freight_value|     order_revenue|
+-----+-------------+------------------+
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
|28.99|         7.46|36.449999999999996|
+-----+-------------+------------------+
only showing top

In [138]:
customer_spending_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- total_orders: long (nullable = false)
 |-- total_spent: double (nullable = true)
 |-- AOV: double (nullable = true)



In [140]:
# Customer Segmentation based on spending

customer_spending_df = customer_spending_df.withColumn(
    'customer_segment',
    when(col('AOV') >=1200,"High-Value")
    .when( (col('AOV')<1200) & (col('AOV') >=700),'Medium_Value')
    .otherwise('Low-Value'))


In [141]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+----------------+
|         customer_id|total_orders|       total_spent|    AOV|customer_segment|
+--------------------+------------+------------------+-------+----------------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|    Medium_Value|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|      High-Value|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|      High-Value|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|      High-Value|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|       Low-Value|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|      High-Value|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|    Medium_Value|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|      High-Value|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|      High-Value|
|695476b5848d64ba0...|         687|18205

In [142]:
full_orders_df = full_orders_df.join(customer_spending_df.select('customer_id','customer_segment'),'customer_id',how='left')

In [144]:
full_orders_df.select('customer_id','customer_segment').show()

+--------------------+----------------+
|         customer_id|customer_segment|
+--------------------+----------------+
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
|9388d79b88a9696b7...|       Low-Value|
+--------------------+----------------+
only showing top 20 rows



In [145]:
full_orders_df.select('order_purchase_timestamp').show()

+------------------------+
|order_purchase_timestamp|
+------------------------+
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
|     2018-08-07 23:19:16|
+------------------------+
only showing top 20 rows



In [146]:
#Hourly Order Distribution
full_orders_df = full_orders_df.withColumn('hour_of_day',expr('hour(order_purchase_timestamp)'))


In [147]:
full_orders_df.select('order_purchase_timestamp','hour_of_day').show()

+------------------------+-----------+
|order_purchase_timestamp|hour_of_day|
+------------------------+-----------+
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
|     2018-08-07 23:19:16|         23|
+------------------------+-----------+
only showing top 20 rows



In [148]:
# Weekday vs Weekend Order

full_orders_df = full_orders_df.withColumn('order_day_type',\
                                           when(dayofweek('order_purchase_timestamp').isin(1,7),lit('Weekend')).otherwise(lit('weekday')))

In [149]:
full_orders_df.select('order_purchase_timestamp','order_day_type').show()

+------------------------+--------------+
|order_purchase_timestamp|order_day_type|
+------------------------+--------------+
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
|     2018-08-07 23:19:16|       weekday|
+------------------------+--------

In [150]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [151]:
!hadoop fs -mkdir /data/olist/processed/

mkdir: `/data/olist/processed': File exists


In [152]:
full_orders_df.write.mode('overwrite').parquet('/data/olist/processed')

In [ ]:
!hadoop fs -ls -h /data/olist/processed/